In [ ]:
import cv2              
import numpy as np  
import os                 
from random import shuffle 
from tqdm import tqdm

In [ ]:
# Define Vars

IMG_SIZE = 256
L = 1e-3  #0.001
MODEL_NAME = '{}-{}.model'.format('alexnet-256X256', L)

In [ ]:
grade_1_data  = np.load("train_data_256X256_grade_1.npy")
grade_2_data  = np.load("train_data_256X256_grade_2.npy")
grade_3_data  = np.load("train_data_256X256_grade_3.npy")
grade_4_data  = np.load("train_data_256X256_grade_4.npy")

In [ ]:
# Merge & Shuffle Data
training_data = np.concatenate(
    (grade_1_data, grade_2_data, grade_3_data, grade_4_data), 
    axis=0
)
np.random.shuffle(training_data)

In [ ]:
#just making sure data in suffeled properly and mergerd too
print(len(training_data))
for data in training_data[:10]:
    print(data[1])

In [ ]:
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression


# Reset Graph
tf.reset_default_graph()

#input layer
network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1])

network = conv_2d(network, 96, 11, strides=4, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 256, 5, activation='relu')

network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)

network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 4, activation='softmax')

network = regression(
                network, 
                optimizer='momentum',
                loss='categorical_crossentropy',
                learning_rate=L
            )

# Training
model = tflearn.DNN(
            network, 
            checkpoint_path=MODEL_NAME,
            max_checkpoints=1, 
            tensorboard_verbose=2,
            tensorboard_dir='log'
        )

In [ ]:
train = training_data[:-8000]
test = training_data[-8000:]

print("Training Data Spamle Size: " + str(len(train)))
print("Test Data Spamle Size: " + str(len(test)))

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

In [ ]:
model.fit(
    X, 
    Y, 
    n_epoch=1000, 
    validation_set=0.1, 
    shuffle=True,
    show_metric=True, 
    batch_size=64, 
    snapshot_step=500,
    snapshot_epoch=False, 
    run_id=MODEL_NAME
)

In [ ]:
model.save(MODEL_NAME)